In [124]:
#Bibliotecas
import pandas as pd
from ydata_profiling import ProfileReport

In [125]:
#importando Bases
df_treino = pd.read_csv(r"files\train.csv")
df_teste = pd.read_csv(r"files\test.csv")
df_gender = pd.read_csv(r"files\gender_submission.csv")

**Analise da Base de Treino**

In [126]:
#Faço uma verificação dos dados com ydata_profiling
profile = ProfileReport(df_treino, title="Titanic_Treino")
profile.to_file("titanic_treino.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 111.17it/s]


In [127]:
#Verifico Tipos dos dados
df_treino.dtypes.value_counts()

int64      5
object     5
float64    2
Name: count, dtype: int64

In [128]:
#Verifico valores nulos
df_treino.isnull().sum().sort_values(ascending=False).head(5)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
dtype: int64

**Analise da Base de Treino**

In [129]:
#Faço uma verificação dos dados com ydata_profiling
profile = ProfileReport(df_teste, title="Titanic_Teste")
profile.to_file("titanic_Teste.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 167.13it/s]


In [130]:
#Verifico Tipos dos dados
df_teste.dtypes.value_counts()

object     5
int64      4
float64    2
Name: count, dtype: int64

In [131]:
#Verifico valores nulos
df_teste.isnull().sum().sort_values(ascending=False).head(5)

Cabin          327
Age             86
Fare             1
PassengerId      0
Pclass           0
dtype: int64

- Temos colunas que possuem Valores Nulos no Treino (Nesse caso, precisamos tratar essas colunas apenas na base de dados)

**Tratando Dados**

In [132]:
#Verifico valores nulos
df_treino.isnull().sum().sort_values(ascending=False).head(5)
df_treino.shape

(891, 12)

In [133]:
#Verifico a cardinalidade dos dados
df_treino.nunique().sort_values(ascending=False)

PassengerId    891
Name           891
Ticket         681
Fare           248
Cabin          147
Age             88
SibSp            7
Parch            7
Pclass           3
Embarked         3
Survived         2
Sex              2
dtype: int64

- colunas com alta cardinalidade não vão ajudar o modelo pois podem torna-lo menos generalizavel. Nesse caso, vamos eliminar essas colunas

In [134]:
#Deletando colunas
df_treino = df_treino.drop(columns=["Name", "Ticket", "Cabin"])
df_teste = df_teste.drop(columns=["Name", "Ticket", "Cabin"])

- Coluna Age possui valores vazios e isso não pode ocorrer, para solucionar isso vamos colocar onde está vazio a media de idade dos tripulantes

In [135]:
#Trato os valores vazios da coluna Age nos 2 df's
df_treino.loc[df_treino.Age.isnull(), "Age"] = df_treino.Age.mean()
df_teste.loc[df_teste.Age.isnull(), "Age"] = df_teste.Age.mean()

- Para a base de treino, ainda temos 2 valores vazios na coluna Embarked
- Podemos verificar todos os valores nessa coluna e substituir os valores vazios pela moda

In [136]:
df_treino.Embarked.mode()[0]

'S'

In [137]:
df_treino.loc[df_treino.Embarked.isnull(), "Embarked"] = df_treino.Embarked.mode()[0]

- Para finalizar, para a base de teste ainda temos 1 valor vazio em Fare
- Podemos utilizar a mesma lógica que fizemos para a coluna Age e substituir pela moda.

In [138]:
df_teste.loc[df_teste.Fare.isnull(), "Fare"] = df_teste.Fare.mean()

**Verifico Novamente se Há Valores Vazios**

In [139]:
df_treino.isnull().sum().sort_values(ascending=False).head(5)


PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
dtype: int64

In [140]:
df_teste.isnull().sum().sort_values(ascending=False).head(5)

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
dtype: int64

**Considerando Apenas as Colunas que não são de Texto**

In [141]:
col_treino = df_treino.columns[df_treino.dtypes != 'object']
col_teste = df_teste.columns[df_teste.dtypes != 'object']

df_treino_nr = df_treino.loc[:,col_treino]
df_teste_nr = df_teste.loc[:,col_teste]

**Começo dos modelos de classficação**

In [142]:
# Importando o train_test_split

from sklearn.model_selection import train_test_split    

In [143]:
# Separando a base de treino em X e y

X = df_treino_nr.drop(columns=['PassengerId','Survived'])
y = df_treino.Survived

In [144]:
# Separando em treino e validação

X_treino, X_validacao, y_treino, y_validacao = train_test_split(X, y, test_size=0.33, random_state=42)

- Trabalhando com Arvore de Classificação

In [145]:
# Fazendo a importação
from sklearn import tree

# Criando o classificador
classficacao_ac = tree.DecisionTreeClassifier(random_state=42)

# Fazendo o fit com os dados
classficacao_ac  = classficacao_ac.fit(X_treino,y_treino)

# Fazendo a previsão
y_previsao_ac = classficacao_ac.predict(X_validacao)

- Para o KNeighborsClassifier

In [146]:
# Importando
from sklearn.neighbors import KNeighborsClassifier

# Criando o classificador
classficacao_knn = KNeighborsClassifier(n_neighbors=3)

# Fazendo o fit com os dados
classficacao_knn = classficacao_knn.fit(X_treino,y_treino)

# Fazendo a previsão
y_previsao_knn = classficacao_knn.predict(X_validacao)

- E para a Regressão Logística

In [147]:
# Importando
from sklearn.linear_model import LogisticRegression

# Criando o classificador
classificacao_rl = LogisticRegression(random_state=42)

# Fazendo o fit com os dados
classificacao_rl = classificacao_rl.fit(X_treino,y_treino)

# Fazendo a previsão
y_previsao_rl = classificacao_rl.predict(X_validacao)

**Avaliação esses modelos**

In [148]:
# Importando
from sklearn.metrics import accuracy_score

In [149]:
# Para a árvore
accuracy_score(y_validacao, y_previsao_ac)

0.6169491525423729

In [150]:
# Para o knn
accuracy_score(y_validacao, y_previsao_knn)

0.6542372881355932

In [151]:
# Para a regressão logística
accuracy_score(y_validacao, y_previsao_rl)

0.7254237288135593

**Avaliação de matriz de confusão**

In [152]:
# Importando
from sklearn.metrics import confusion_matrix

In [153]:
# Para a árvore
confusion_matrix(y_validacao, y_previsao_ac)

array([[125,  50],
       [ 63,  57]], dtype=int64)

In [154]:
# Para o knn
confusion_matrix(y_validacao, y_previsao_knn)

array([[133,  42],
       [ 60,  60]], dtype=int64)

In [155]:
# Para a regressão logística
confusion_matrix(y_validacao, y_previsao_rl)

array([[156,  19],
       [ 62,  58]], dtype=int64)

**Previsão para com os Dados de Teste**

In [156]:
# Visualizando o X_treino

X_treino.head(3)

,Pclass,Age,SibSp,Parch,Fare
6,1,54.000000,0,0,51.8625
718,3,29.699118,0,0,15.5000
685,2,25.000000,1,2,41.5792


In [157]:
# Visualizando a base de teste (apenas com valores numéricos)

df_teste_nr.head(3)

,PassengerId,Pclass,Age,SibSp,Parch,Fare
0,892,3,34.5,0,0,7.8292
1,893,3,47.0,1,0,7.0000
2,894,2,62.0,0,0,9.6875


In [158]:
# Para a base de teste ser igual a base de treino, precisamos eliminar a coluna de id
X_teste = df_teste_nr.drop("PassengerId",axis=1)

In [159]:
# Utilizando a regressão logística na base de teste
y_previsao = classificacao_rl.predict(X_teste)

In [ ]:
# Criando uma coluna com a previsão na base de teste
teste['Survived'] = y_previsao

In [ ]:
# Selecionando apenas a coluna de Id e Survived para fazer o envio
df_base_envio = teste[['PassengerId','Survived']]